In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
%matplotlib inline
import ase
import subprocess
import re

In [62]:
def apply_pbc_distance(df, PKA, radius, xlo, xhi, ylo, yhi, zlo, zhi):
    """
    Apply periodic boundary conditions to calculate the minimum image distance 
    between each atom and the PKA, then filter atoms within a given radius.
    
    Parameters:
        df (pd.DataFrame): DataFrame with atom coordinates.
        PKA (pd.Series): Series representing the PKA coordinates.
        radius (float): The cutoff radius within which atoms are selected.
        xlo, xhi, ylo, yhi, zlo, zhi (float): Box dimensions.

    Returns:
        pd.DataFrame: Filtered DataFrame containing atoms within the radius.

    """
    # Calculate box lengths for each dimension
    box_x = xhi - xlo
    box_y = yhi - ylo
    box_z = zhi - zlo

    # Compute minimum image distance in each direction
    dx = np.abs(df['x'] - PKA['x'])
    dx = np.where(dx > box_x / 2, box_x - dx, dx)

    dy = np.abs(df['y'] - PKA['y'])
    dy = np.where(dy > box_y / 2, box_y - dy, dy)

    dz = np.abs(df['z'] - PKA['z'])
    dz = np.where(dz > box_z / 2, box_z - dz, dz)

    # Calculate the total distance with PBC applied
    distances = np.sqrt(dx**2 + dy**2 + dz**2)

    # Filter atoms within the specified radius
    dff = df[distances <= radius]

    return dff

In [63]:
def spherical_to_cartesian(r, phi, theta):
    """ convert spherical coordinates (physics convention) to cart. vector
        input angles in DEGREES! """
    phi = np.deg2rad(phi)
    theta = np.deg2rad(theta)
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return np.array([x, y, z])

def random_direction(seed=None):
    """ generate random direction in spherical coordinates """
    np.random.seed(seed)  # when None, seeds from /dev/urandom if available, else clock
    theta = np.arccos(2.0 * np.random.random() - 1)
    phi = 2.0 * np.random.random() * np.pi
    theta = np.rad2deg(theta)
    phi = np.rad2deg(phi)
    return phi, theta

In [64]:
def displacement(fpas,xlo, xhi, ylo, yhi, zlo, zhi):
    """
    This function takes a DataFrame `fpas` and applies a random displacement 
    directly to the `x`, `y`, and `z` coordinates of each particle. If a displaced 
    position goes outside the box boundaries, it is wrapped back inside using 
    periodic boundary conditions.
    """
    # Reset the index to ensure we have a sequential numeric index
    fpas = fpas.reset_index(drop=True)
    # Box dimensions
    x_length = xhi - xlo
    y_length = yhi - ylo
    z_length = zhi - zlo
    for j in range(fpas.shape[0]):
        # Generate a random displacement distance within a specific range
        r = np.random.uniform(5.8, 6.2)
        
        # Generate random spherical coordinates for direction
        phi, theta = random_direction()
        
        # Convert spherical coordinates to Cartesian displacements
        displace_x, displace_y, displace_z = spherical_to_cartesian(r, phi, theta)
        # Calculate new positions with displacement
        new_x = fpas.at[j, 'x'] + displace_x
        new_y = fpas.at[j, 'y'] + displace_y
        new_z = fpas.at[j, 'z'] + displace_z
        
        # Apply periodic boundary conditions for each axis
        if new_x < xlo:
            new_x += x_length
        elif new_x > xhi:
            new_x -= x_length
            
        if new_y < ylo:
            new_y += y_length
        elif new_y > yhi:
            new_y -= y_length
            
        if new_z < zlo:
            new_z += z_length
        elif new_z > zhi:
            new_z -= z_length
        
        # Update the DataFrame with the new wrapped positions
        fpas.at[j, 'x'] = new_x
        fpas.at[j, 'y'] = new_y
        fpas.at[j, 'z'] = new_z

    return fpas
def apply_displacement_to_df(df, ofpas):
    """
    Apply the updated positions from `ofpas` to the corresponding atoms in `df`.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame containing all atoms.
        ofpas (pd.DataFrame): The DataFrame containing the displaced atoms.
        
    Returns:
        pd.DataFrame: The modified DataFrame with updated positions.
    """
    for index, row in ofpas.iterrows():
        # Extract the Particle Identifier and the updated positions
        particle_id = row['Particle Identifier']
        new_x = row['x']  # Use the updated position
        new_y = row['y']
        new_z = row['z']

        # Locate the corresponding atom in the original df using Particle Identifier
        atom_index = df[df['Particle Identifier'] == particle_id].index
        
        if len(atom_index) > 0:  # Ensure the atom is found
            # Update the positions in df
            df.at[atom_index[0], 'x'] = new_x
            df.at[atom_index[0], 'y'] = new_y
            df.at[atom_index[0], 'z'] = new_z

    return df            

In [65]:
def write_lammps_data_file(df, box_dimensions, output_file):
    """
    Write a LAMMPS data file with updated positions from the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing updated atom positions.
        box_dimensions (tuple): The dimensions of the box (xlo, xhi, ylo, yhi, zlo, zhi).
        output_file (str): The path to the output LAMMPS data file.
    """
    xlo, xhi, ylo, yhi, zlo, zhi = box_dimensions

    with open(output_file, 'w') as f:
        # Header
        f.write("LAMMPS data file via write_data, version 3 Aug 2022\n\n")
        f.write(f"{len(df)} atoms\n")
        f.write("5 atom types\n\n")
        f.write(f"{xlo} {xhi} xlo xhi\n")
        f.write(f"{ylo} {yhi} ylo yhi\n")
        f.write(f"{zlo} {zhi} zlo zhi\n\n")
        
        # Masses
        f.write("Masses\n\n")
        f.write("1 69.723\n")
        f.write("2 69.723\n")
        f.write("3 15.999\n")
        f.write("4 15.999\n")
        f.write("5 15.999\n\n")
        
        # Atoms
        f.write("Atoms # atomic\n\n")
        for index, row in df.iterrows():
            f.write(f"{int(row['Particle Identifier'])} {int(row['Particle Type'])} "
                     f"{row['x']} {row['y']} {row['z']} 0 0 0\n")

In [66]:
datafile='/Users/ruhe/Desktop/ga2o3/rbs/FPs_cc_misorientation/data.8_64_34'
with open(datafile, 'r') as file:
    
    for line in file:
            # Search for xlo xhi, ylo yhi, and zlo zhi lines using regex
        x_match = re.match(r"(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+xlo xhi", line)
        y_match = re.match(r"(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+ylo yhi", line)
        z_match = re.match(r"(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+zlo zhi", line)
        
        if x_match:
            xlo, xhi = float(x_match.group(1)), float(x_match.group(2))
        elif y_match:
            ylo, yhi = float(y_match.group(1)), float(y_match.group(2))
        elif z_match:
            zlo, zhi = float(z_match.group(1)), float(z_match.group(2))

    # Display extracted values
print("xlo:", xlo, "xhi:", xhi)
print("ylo:", ylo, "yhi:", yhi)
print("zlo:", zlo, "zhi:", zhi)



xlo: -0.2925932898 xhi: 194.0057474571
ylo: -0.2982921814 yhi: 395.8671560025
zlo: -0.3019367948 zhi: 200.2010148527


In [56]:

df=pd.read_csv(datafile, skiprows = 19,sep="\s+",header=None,nrows=696320)
df = df.drop(df.columns[-3:], axis=1)
df.columns=['Particle Identifier','Particle Type','x','y','z']
df = df.apply(lambda col: pd.to_numeric(col, errors='coerce'))
#two center:(50,50,150) and (150,150,50)
#find df radius r=15 
radius=80
centers = [pd.Series({'x': 50, 'y': 50, 'z': 50}),pd.Series({'x': 150, 'y': 150, 'z': 150}),pd.Series({'x': 50, 'y': 250, 'z': 150}),pd.Series({'x': 150, 'y': 350, 'z': 50})]
for center in centers:
    dff1 = apply_pbc_distance(df, center , radius, xlo, xhi, ylo, yhi, zlo, zhi)
    fpga1=dff1[dff1['Particle Type']<3]

    dfga1=displacement(fpga1,xlo, xhi, ylo, yhi, zlo, zhi)
    df=apply_displacement_to_df(df, dfga1)




,Particle Identifier,Particle Type,x,y,z
0,20563,1,24.699193,192.392636,12.422037
1,677764,1,3.057280,19.454744,194.525318
2,1,2,190.814362,2.123816,3.929347
3,34,3,1.598458,2.779153,-0.229027
4,31,5,1.890193,2.874266,2.450289
...,...,...,...,...,...
696315,20409,4,175.756621,191.567858,200.118915
696316,20442,2,183.051714,189.763825,195.237480
696317,696299,3,191.853280,191.618350,199.996764
696318,19082,2,157.557568,182.880641,198.723735


In [57]:
box_dimensions = (xlo, xhi, ylo, yhi, zlo, zhi)  # You should define these dimensions
output_file='/Users/ruhe/Desktop/ga2o3/rbs/FPs_cc_misorientation/data_40nm'
write_lammps_data_file(df, box_dimensions, output_file)

In [60]:
datafile='/Users/ruhe/Desktop/ga2o3/rbs/FPs_cc_misorientation/data.relaxed8_32_34'
df=pd.read_csv(datafile, skiprows = 19,sep="\s+",header=None,nrows=696320)
df = df.drop(df.columns[-3:], axis=1)
df.columns=['Particle Identifier','Particle Type','x','y','z']
df = df.apply(lambda col: pd.to_numeric(col, errors='coerce'))

dff1 =df[df['x']<100]
fpga1=dff1[dff1['Particle Type']<3]
#fpga1=df[df['Particle Type']<3]
dfga1=displacement(fpga1,xlo, xhi, ylo, yhi, zlo, zhi)
df=apply_displacement_to_df(df, dfga1)
box_dimensions = (xlo, xhi, ylo, yhi, zlo, zhi)  # You should define these dimensions
output_file='/Users/ruhe/Desktop/ga2o3/rbs/FPs_cc_misorientation/data_slicex'
write_lammps_data_file(df, box_dimensions, output_file)